# Project Pipeline

Lucovica Schaerf, Antònio Mendes, Jaël Kortekaas

Large part of our code is used from: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

## Introduction

This file contains the the preprocessing pipeline to our project. 
As a first step we are importing the data and filtering out all the songs that
we don't need for our analysis. Secondly, we will implement the 'standard' 
pipeline and, once we obtain the most common words per each album, author, year
(...) we will move to another file to do the clustering and topic analysis.

## Import

In [6]:
from pathlib import Path
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.collocations import *
from nltk.collocations import BigramAssocMeasures
import nltk
import re
import string
import gensim
import spacy

from gensim import corpora, models
from nltk.corpus import wordnet as wn
from operator import itemgetter
import itertools

import matplotlib.pyplot as plt

plot_dir = Path("./figures")
data_dir = Path("./data")

import sklearn
from sklearn import metrics, manifold
import scipy
from scipy import cluster
import matplotlib as mpl 
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

In [7]:
songs = []

with open('./data/lyrics.csv', 'r', encoding="utf-8") as infile:
    songs = pd.read_csv(infile)
    
print(songs.columns)

Index(['index', 'song', 'year', 'artist', 'genre', 'lyrics'], dtype='object')


In [8]:
artists = ['bruce-springsteen', 'elliot-smith', 'black-sabbath', 'deep-purple', 'david-bowie']

david_bowie = songs[songs[u'artist'] == 'david-bowie']
black_sabbath = songs[songs[u'artist'] == 'black-sabbath']
bruce_springsteen = songs[songs[u'artist'] == 'bruce-springsteen']
elliot_smith = songs[songs[u'artist'] == 'elliot-smith']
deep_purple = songs[songs[u'artist'] == 'deep-purple']

lyrics = pd.concat([david_bowie, black_sabbath, bruce_springsteen, elliot_smith, deep_purple], axis=0)
lyrics = lyrics.dropna()
lyrics

,index,song,year,artist,genre,lyrics
116793,116793,if-i-m-dreaming-my-life,2009,david-bowie,Rock,VERSE (there)\nWas she never there/here?\nWas ...
116794,116794,seven,2009,david-bowie,Rock,I forgot what my father said\nI forgot what he...
116795,116795,i-can-t-read,2009,david-bowie,Rock,I can't read and I can't write down\nI don't k...
116796,116796,thursday-s-child,2009,david-bowie,Rock,All of my life I've tried so hard\nDoing my be...
116797,116797,survive,2009,david-bowie,Rock,"Oh, my\nNaked eyes\nI should have kept you\nI ..."
...,...,...,...,...,...,...
67509,67509,child-in-time,1972,deep-purple,Rock,Sweet child in time\nYou'll see the line\nThe ...
67510,67510,deep-purple-overture,2015,deep-purple,Rock,"Good golly, said little miss molly\nWhen she w..."
67517,67517,paint-it-black,2015,deep-purple,Rock,"I see a red door and I want it painted black,\..."
67523,67523,anyone-s-daughter,2014,deep-purple,Rock,"Well, I stood under your bedroom window, throw..."


## Processing Pipeline

In [9]:
stop_words = stopwords.words('english')
stop_words.extend(['oh', 'yeah', 'hey', 'doo', 'oo', 'uh', 'la', 'verse', 'chorus', 'bridge', 'x2', "'m"]) # filter out common meaningless words/sounds and words describing song structure

The preprocessing pipeline we decided to implement includes the following steps:
- filtering out stopwords, puntuation, sounds typical from songs
- lemmatizing
- adding the most common bigrams

In [10]:
wnl = WordNetLemmatizer()
bigram_measures = BigramAssocMeasures()

def convertTuple(tup): 
    str =  '_'.join(tup) 
    return str
  
def simple_preprocess(lyrics, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    lyricslist = [re.sub('\-', '', str(lyric)) for lyric in lyrics['lyrics'].tolist()] # take out all hyphens that often connect meaningless words/sounds to these stopwords can be filtered out later
    lyricslist = [re.sub('[\.\,\?,\!,\(,\),\:,\"]', '', str(lyric)) for lyric in lyricslist] # take out all punctuation
    lyricslist = [word_tokenize(lyric.lower()) for lyric in lyricslist]
    finder = []
    for i,lyric in enumerate(lyricslist):
        for j,token in enumerate(lyric):
            lyricslist[i][j] = wnl.lemmatize(str(token))
        lyricslist[i] = [word for word in lyric if word not in stop_words]
        finder.append(BigramCollocationFinder.from_words(lyric))
        finder[i] = finder[i].nbest(bigram_measures.pmi, 20)
        finder[i] = [convertTuple(x) for x in finder[i]] #need to append the two words back together
        lyricslist[i] = lyricslist[i] + finder[i]
    lyrics['bag_of_words'] = lyricslist


simple_preprocess(lyrics)
example = lyrics['bag_of_words'][116793]
example

['wa',
 'never',
 'there/here',
 'wa',
 'ever',
 'wa',
 'air',
 'breathed',
 'wrong',
 'time',
 'ohoh',
 'ohoh',
 'flower',
 'gallery',
 'hymn',
 'night',
 'singing',
 'come',
 'wrong',
 'time',
 'wrong',
 'day',
 'light',
 'fading',
 'dreaming',
 'life',
 'second',
 'time',
 'one',
 'living',
 'chance',
 'mother',
 'sigh',
 'father',
 'step',
 'aside',
 'wrong',
 'time',
 'ohoh',
 'wa',
 'ever',
 'wa',
 'ever',
 'dreaming',
 'life',
 'dreaming',
 'life',
 'away',
 'dreaming',
 'life',
 'dreaming',
 'dreaming',
 'dreaming',
 'dreaming',
 'life',
 'dreaming',
 'life',
 'away',
 'ohoh',
 'repeat',
 'dreaming',
 'life',
 'dreaming',
 'dreaming',
 'life',
 'away',
 'ohoh',
 'are_fading',
 'come_to',
 'fading_now',
 'father_step',
 'flower_so',
 'gallery_with',
 'hymn_of',
 'it_air',
 'just_one',
 'light_are',
 'living_chance',
 'mother_sigh',
 'never_there/here',
 'night_singing',
 'of_night',
 'one_living',
 'singing_come',
 'so_from',
 'step_aside',
 'to_me']

# 20 common topics between all songs by all artits
This first part is an attempt to identify some common topics, in order to have a baseline of
general musical topics that often appear. 

In [26]:
#count how many times a word appears --> give a document
#lyrics['bag_of_words']

list_BOWlyrics = [[]]
for lyric in lyrics['bag_of_words']:
    list_BOWlyrics += [lyric]

lyrics_dictionary = corpora.Dictionary(list_BOWlyrics)
print('Number of unique tokens:', len(lyrics_dictionary))

Number of unique tokens: 31189


In [27]:
print(dict(itertools.islice(lyrics_dictionary.token2id.items(), 12)))
print("word with id 8:", lyrics_dictionary[8])
print("frequency of token 8:", lyrics_dictionary.dfs[8])

{'air': 0, 'are_fading': 1, 'aside': 2, 'away': 3, 'breathed': 4, 'chance': 5, 'come': 6, 'come_to': 7, 'day': 8, 'dreaming': 9, 'ever': 10, 'fading': 11}
word with id 8: day
frequency of token 8: 415


In [28]:
# Filter out words that occur in less than 5 documents, or more than 70% of the documents.
lyrics_dictionary.filter_extremes(no_below=2, no_above=0.7)
print('Number of unique tokens:', len(lyrics_dictionary))

Number of unique tokens: 3046


In [29]:
# Bag-of-words representation of the documents
lyrics_bow_corpus = [lyrics_dictionary.doc2bow(d) for d in list_BOWlyrics]

In [35]:
# the BOW representation of the first document
print(lyrics_bow_corpus[1][:50])

[(0, 1), (1, 1), (2, 3), (3, 1), (4, 1), (5, 1), (6, 12), (7, 3), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 8), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 4), (26, 5), (27, 4)]


In [33]:
# which words (and how often) appear in the first document?
for i, freq in sorted(lyrics_bow_corpus[1], key=itemgetter(1), reverse=True)[:15]:
    print(lyrics_dictionary[i], "-->", freq)
print("...")

dreaming --> 12
life --> 8
wa --> 5
time --> 4
wrong --> 4
away --> 3
ever --> 3
air --> 1
aside --> 1
chance --> 1
come --> 1
day --> 1
fading --> 1
father --> 1
flower --> 1
...


In [39]:
lyrics_ldamodel = models.ldamodel.LdaModel(lyrics_bow_corpus, num_topics=20, id2word = lyrics_dictionary, passes= 25)

In [40]:
# the setting formatted=False allows you to get rid of the word*probability format when retrieveing topics
lyrics_ldamodel.show_topics(formatted=False, num_words=15)

[(1,
  [('long', 0.03684884),
   ('home', 0.028831301),
   ('back', 0.02327016),
   ('wa', 0.02144347),
   ('never', 0.019309334),
   ('bring', 0.017147055),
   ("'ll", 0.016577782),
   ("'ve", 0.011354292),
   ('go', 0.010860615),
   ('fire', 0.010695467),
   ('ah', 0.010335342),
   ('eye', 0.010149982),
   ("'em", 0.0100763),
   ('town', 0.009408002),
   ("'d", 0.009140688)]),
 (8,
  [('take', 0.05732043),
   ("n't", 0.03149467),
   ('come', 0.027449243),
   ('right', 0.021535905),
   ("'re", 0.016061736),
   ('hand', 0.015427335),
   ('heart', 0.012536857),
   ('shine', 0.012062132),
   ('love', 0.0116058355),
   ('gon', 0.011558871),
   ('na', 0.010237374),
   ('see', 0.010018136),
   ('say', 0.009503205),
   ('rise', 0.009429461),
   ('know', 0.008621004)]),
 (19,
  [('ba', 0.030092109),
   ('love', 0.026689498),
   ('people', 0.01832417),
   (']', 0.018083861),
   ('[', 0.018014856),
   ('man', 0.01770162),
   ('instrumental', 0.016124912),
   ('lie', 0.014542432),
   ('telling',

# 20 common topics between all songs per artist
Now we are interested in identifying what are the common topics of each artists

In [47]:
#count how many times a word appears --> give a document
#lyrics['bag_of_words']
list_BOWlyrics_per_artist = [] 
lyrics_dictionary_per_artist = []

for i,artist in enumerate(artists):
    lyrics_of_artist = []
    for lyric in lyrics[lyrics['artist'] == artist]['bag_of_words']:
        lyrics_of_artist += [lyric]
    list_BOWlyrics_per_artist.append(lyrics_of_artist)

for i,artist in enumerate(artists):
    lyrics_dictionary_per_artist.append(corpora.Dictionary(list_BOWlyrics_per_artist[i]))
    print('Number of unique tokens:', len(lyrics_dictionary_per_artist[i]))

Number of unique tokens: 14039
Number of unique tokens: 1690
Number of unique tokens: 5423
Number of unique tokens: 6415
Number of unique tokens: 13681


In [48]:
lyrics_bow_corpus_per_artist = []

for i,artist in enumerate(artists):
    lyrics_bow_corpus_per_artist.append([lyrics_dictionary_per_artist[i].doc2bow(d) 
                                         for d in list_BOWlyrics_per_artist[i]])
    

In [49]:
lyrics_ldamodel_per_artist = []

for i,artist in enumerate(artists):
    lyrics_ldamodel_per_artist.append(models.ldamodel.LdaModel(lyrics_bow_corpus_per_artist[i],\
                                                               num_topics=20, \
                                                               id2word = lyrics_dictionary_per_artist[i],\
                                                               passes= 25))


In [54]:
# the setting formatted=False allows you to get rid of the word*probability format when retrieveing topics
for i,artist in enumerate(artists):
    print(artist, lyrics_ldamodel_per_artist[i].show_topics(formatted=False, num_words=15), '\n')

bruce-springsteen [(15, [('train', 0.034895256), ('give', 0.020677026), ('got', 0.018006422), ("n't", 0.016367648), ('hope', 0.015394403), ('love', 0.014963676), ('ooh', 0.0139214), ('u', 0.013627458), ('fun', 0.012805798), ('may', 0.012710164), ('carry', 0.011292292), ("'ll", 0.010942565), ('come', 0.009879748), ('take', 0.009471915), ('get', 0.008475069)]), (7, [('ho', 0.019080045), ('wa', 0.012753229), ('santa', 0.012539801), ('coming', 0.011752582), ('man', 0.011267436), ('town', 0.0111444155), ('claus', 0.010804578), ('sha', 0.010797463), ('heart', 0.009613637), ("n't", 0.009346355), ("'", 0.008933277), ("'s", 0.0084844185), ('come', 0.007041963), ('like', 0.006693133), ('sold', 0.006687723)]), (19, [("'", 0.045649007), ('bring', 0.018815724), ('home', 0.015046288), ("'em", 0.014603241), ('li', 0.012171351), ('thinkin', 0.012170738), ('day', 0.010068078), ('wa', 0.009608982), ("'s", 0.0093703475), ('girl', 0.007986004), ('baby', 0.00751635), ('sandy', 0.00734085), ("n't", 0.006064